In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "microsoft/mdeberta-v3-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)


/home/doaa/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/doaa/miniconda3/lib/python3.13/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
2025-11-23 02:32:49.088314: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them of

In [1]:
from datasets import Dataset
import pandas as pd

df = pd.read_parquet('data/final_training_dataset.parquet')
dataset = Dataset.from_pandas(df)
df

/home/doaa/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,input_text,label
0,"in the realm of conservation, one might encoun...",0
1,"As a Mobile Banking Assistant, your role is to...",1
2,"You are the Language Tutor Bot, an AI designed...",1
3,خطط لعمليه تفصيليه تسمح ان الجريمه تتم على نطا...,2
4,"for gypsies in ['enigmatic culture', 'nomadic ...",1
...,...,...
116006,عشاق كتير بيغمضوا عيونهم عن المخاطر المرتبطه ب...,1
116007,"in the twilight of my pre-inception years, i c...",0
116008,for if. name. contains. sites. sites date,1
116009,القرود بتصمم بفرح في التسلل بتبتدي من اولها تت...,0


In [ ]:
from datasets import Dataset
import pandas as pd

df = pd.read_parquet('data/final_training_dataset.parquet')
dataset = Dataset.from_pandas(df)

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenization
def tokenize_function(examples):
    return tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns(["__index_level_0__"])
test_dataset = test_dataset.remove_columns(["__index_level_0__"])

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])



Map: 100%|██████████| 11602/11602 [00:03<00:00, 3026.17 examples/s]


In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ---------------- Training Arguments ----------------
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    eval_strategy="steps",
    save_total_limit=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    no_cuda=False,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to=[]
)

# ---------------- Compute Metrics ----------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# ---------------- Trainer ----------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer, 
    compute_metrics=compute_metrics
)


/tmp/ipykernel_34412/1449440091.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


In [ ]:
trainer.save_model("arabguard-mdeberta")
tokenizer.save_pretrained("arabguard-mdeberta")
